**Dicionário catalão COVID**: 

Passar as termos chave para os termos em português

Ficheiro de saída: dicionario_medico_covid_pt.json

In [30]:
import json
import os

# Carregar o JSON catalão
with open("../dici_covid_corrigido/dicionario_covid_corrigido.json", "r", encoding="utf-8") as f:
    dados = json.load(f)

novos_dados = {}

for termo_original, info in dados.items():
    traducoes = info.get("Traduções", {})
    termo_pt = None

    # Verifica se há uma tradução em pt_PT com prioridade (porque pode haver pt_PT e pt na mesma entrada)
    if "pt_PT" in traducoes:
        termo_pt = traducoes["pt_PT"][0]
    elif "pt" in traducoes and "pt_PT" not in traducoes:
        termo_pt = traducoes["pt"][0]

    if termo_pt:
        nova_chave = termo_pt

        # Clona a entrada
        nova_entrada = info.copy()

        # Atualiza traduções
        novas_traducoes = nova_entrada.get("Traduções", {}).copy()
        novas_traducoes.pop("pt", None)
        novas_traducoes.pop("pt_PT", None)
        novas_traducoes["ca"] = [termo_original]
        nova_entrada["Traduções"] = novas_traducoes

        # Se já existe uma entrada com a nova chave, mesclar os dados
        if nova_chave in novos_dados:
            print(f"Aviso: A chave '{nova_chave}' já existe. As informações serão combinadas.")
            entrada_existente = novos_dados[nova_chave]

            entrada_fundida = {}

            for campo in set(entrada_existente) | set(nova_entrada):
                val1 = entrada_existente.get(campo)
                val2 = nova_entrada.get(campo)

                if isinstance(val1, list) and isinstance(val2, list):
                    entrada_fundida[campo] = list(set(val1 + val2))
                elif isinstance(val1, dict) and isinstance(val2, dict):
                    fundido = {}
                    for k in set(val1) | set(val2):
                        v1 = val1.get(k, [])
                        v2 = val2.get(k, [])
                        if isinstance(v1, list) and isinstance(v2, list):
                            fundido[k] = list(set(v1 + v2))
                        else:
                            fundido[k] = v2 or v1
                    entrada_fundida[campo] = fundido
                else:
                    entrada_fundida[campo] = val2 if val2 is not None else val1

            novos_dados[nova_chave] = entrada_fundida
        else:
            novos_dados[nova_chave] = nova_entrada
    else:
        # Se não há tradução em pt, mantém com o termo original
        novos_dados[termo_original] = info

# Salvar
with open("dicionarios_originais/dicionario_medico_covid_pt.json", "w", encoding="utf-8") as f:
    json.dump(novos_dados, f, ensure_ascii=False, indent=2)

print("Transformação concluída. Arquivo salvo como 'dicionario_medico_covid_pt.json'")


Aviso: A chave 'incapacidade' já existe. As informações serão combinadas.
Aviso: A chave 'RAV' já existe. As informações serão combinadas.
Aviso: A chave 'semiologia' já existe. As informações serão combinadas.
Aviso: A chave 'CVnCoV' já existe. As informações serão combinadas.
Transformação concluída. Arquivo salvo como 'dicionario_medico_covid_pt.json'


**Juntar de novo todos os ficheiros**

In [31]:
import json

# Função para carregar ficheiros
def load_json(caminho):
    with open(caminho, encoding="utf-8") as f:
        return json.load(f)

# Função básica para fundir dois valores
def merge(v1, v2):
    if type(v1) == dict and type(v2) == dict:
        resultado = {}
        for chave in set(list(v1.keys()) + list(v2.keys())):
            if chave in v1 and chave in v2:
                resultado[chave] = merge(v1[chave], v2[chave])
            elif chave in v1:
                resultado[chave] = v1[chave]
            else:
                resultado[chave] = v2[chave]
        return resultado
    
    elif type(v1) == list and type(v2) == list:
        return list(set(v1 + v2))
    
    elif type(v1) == str and type(v2) == str:
        if v1 == v2:
            return v1
        else:
            return list(set([v1, v2]))
    
    elif type(v1) == list:
        return list(set(v1 + [v2]))
    
    elif type(v2) == list:
        return list(set([v1] + v2))
    
    else:
        return list(set([v1, v2]))

# Carregamento dos dicionários
d1 = load_json("dicionarios_originais/dicionario_medico_covid_pt.json")
d2 = load_json("dicionarios_originais/glossario.json")
d3 = load_json("dicionarios_originais/termos_medicos_populares.json")
d4 = load_json("dicionarios_originais/monitoramento-e-avaliacao-glossario.json")

# Lista com todos os dicionários
todos = [d1, d2, d3, d4]

# Dicionário final
unificado = {}

for dicionario in todos:
    for termo in dicionario:
        if termo not in unificado:
            unificado[termo] = dicionario[termo]
        else:
            unificado[termo] = merge(unificado[termo], dicionario[termo])

# Guardar o resultado final
with open("dicionario_final_pt.json", "w", encoding="utf-8") as f_out:
    json.dump(unificado, f_out, ensure_ascii=False, indent=2)


In [6]:
d_TP1 = load_json("dicionarios_originais/dicionario_final.json")
d_novo = load_json("dicionario_final_pt.json")

print (f"Total de termos no dicionário da TP1: {len(d_TP1)}")
print(f"Total de termos no dicionário novo pt: {len(d_novo)}")

Total de termos no dicionário da TP1: 3085
Total de termos no dicionário novo pt: 3035


Extrair termos de um site sobre a covid: https://www.jpn.up.pt/2020/04/07/covid-19-as-palavras-que-entraram-no-nosso-vocabulario/

dicionario_atualizado1.json

In [32]:
import requests
from bs4 import BeautifulSoup
import re
import json

# URL da página (substitua pela URL real, caso esteja acessando diretamente)
url = "https://www.jpn.up.pt/2020/04/07/covid-19-as-palavras-que-entraram-no-nosso-vocabulario/"  # Insira a URL real da página aqui

# Função para limpar texto removendo quebras de linha e espaços extras
def clean_text(text):
    return re.sub(r'\s+', ' ', text.strip())

# Fazer a requisição HTTP para obter o conteúdo da página
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Encontrar a seção com a classe 'post-content'
post_content = soup.find('section', class_='post-content')

# Lista para armazenar os resultados
glossary = []

# Encontrar todas as listas <ul> dentro da seção
ul_elements = post_content.find_all('ul')

for ul in ul_elements:
    # Encontrar todos os itens <li> dentro de cada <ul>
    li_elements = ul.find_all('li')
    for li in li_elements:
        # Extrair o termo, que está em <strong> ou <em><strong>
        strong = li.find('strong')
        if strong:
            term = clean_text(strong.get_text())
            # Extrair a definição, que está em <em> após o termo
            em = li.find('em')
            definition = clean_text(em.get_text()) if em else ''
            # Extrair o texto relacionado, que é o conteúdo após o <em> ou todo o texto do <li>
            related_text = clean_text(li.get_text().replace(strong.get_text(), '').replace(em.get_text() if em else '', ''))
            glossary.append({
                'term': term,
                'definition': definition,
                'related_text': related_text
            })

# Salvar os resultados em um arquivo JSON
os.makedirs('dicionarios_scrapping', exist_ok=True)
with open('dicionarios_scrapping/covid_glossary.json', 'w', encoding='utf-8') as f:
    json.dump(glossary, f, ensure_ascii=False, indent=4)

print("Extração concluída. Resultados salvos em 'covid_glossary.json'.")

Extração concluída. Resultados salvos em 'covid_glossary.json'.


In [33]:
import json

# Carregar os dados
with open("dicionario_final_pt.json", "r", encoding="utf-8") as f:
    termos_convertidos = json.load(f)

with open("dicionarios_scrapping/covid_glossary.json", "r", encoding="utf-8") as f_covid_pt:
    glossary_covid_pt = json.load(f_covid_pt)

# Extrair os termos do glossário e converter para minúsculas
glossary_terms_covid = {item["term"].lower() for item in glossary_covid_pt}

# Extrair os termos do outro dicionário (já são as chaves)
other_terms = {term.lower() for term in termos_convertidos.keys()}

# Encontrar termos em comum
common_terms = glossary_terms_covid.intersection(other_terms)

# Exibir os termos em comum e detalhes
if common_terms:
    print("Termos em comum encontrados:")
    for term in common_terms:
        # Buscar os dados originais no glossário
        for item in glossary_covid_pt:
            if item["term"].lower() == term:
                
                print(f"{term}")
                # print(f"Definição (do web scraping): {item.get('definition', 'N/A')}")
                # print(f"Texto Relacionado (do web scraping): {item.get('related_text', 'N/A')}")
                # print(f"Dados no outro dicionário: {termos_convertidos.get(term, 'N/A')}")
                # print("-" * 50)
        print()
else:
    print("Nenhum termo em comum encontrado.")

Termos em comum encontrados:
quarentena

fase de mitigação

ventilador

pandemia

sintomatologia

imunidade comunitária

coronavírus

teletrabalho

zaragatoa

viseira



In [34]:
import json

# Carregar os dados
with open("dicionario_final_pt.json", "r", encoding="utf-8") as f:
    dicionario_pt = json.load(f)

with open("dicionarios_scrapping/covid_glossary.json", "r", encoding="utf-8") as f_covid_pt:
    glossary_covid_pt = json.load(f_covid_pt)

# Criar um dicionário de termos do glossário para acesso rápido
glossary_dict = {
    item["term"].lower(): item for item in glossary_covid_pt
}

dicionario_pt_lower = {k.lower(): k for k in dicionario_pt}

# Atualizar os termos existentes e adicionar os novos
for termo_lower, item in glossary_dict.items():
    descricao_completa = f"{item.get('definition', '').strip()} {item.get('related_text', '').strip()}"
    
    if termo_lower in dicionario_pt_lower:
        termo_chave = dicionario_pt_lower[termo_lower]
        dicionario_pt[termo_chave]["descricao_pt"] = descricao_completa
    else:
        dicionario_pt[item["term"]] = {
            "descricao_pt": descricao_completa
        }

# Gravar em novo ficheiro para preservar o original
os.makedirs("dicionarios_atualizados", exist_ok=True)
with open("dicionarios_atualizados/dicionario_atualizado1.json", "w", encoding="utf-8") as f_out:
    json.dump(dicionario_pt, f_out, ensure_ascii=False, indent=2)

print("Atualização concluída. Ficheiro salvo como 'dicionario_atualizado1.json'.")


Atualização concluída. Ficheiro salvo como 'dicionario_atualizado1.json'.


In [35]:
with open("dicionarios_atualizados/dicionario_atualizado1.json", "r", encoding="utf-8") as f:
    dicionario1 = json.load(f)
print(f"Total de entradas no glossario atualizado1: {len(dicionario1)}") 

# with open("dicionarios_atualizados/dicionario_atualizado11.json", "r", encoding="utf-8") as f:
#     dicionario11 = json.load(f)
# print(f"Total de entradas no glossario atualizado11: {len(dicionario11)}")

# 3035 -> 3053

Total de entradas no glossario atualizado1: 3053
